# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [99]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import sys

['', '/opt/spark-2.4.3-bin-hadoop2.7/python', '/opt/spark-2.4.3-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip', '/home/workspace', '/opt/conda/lib/python36.zip', '/opt/conda/lib/python3.6', '/opt/conda/lib/python3.6/lib-dynload', '/opt/conda/lib/python3.6/site-packages', '/opt/conda/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg', '/opt/conda/lib/python3.6/site-packages/cycler-0.10.0-py3.6.egg', '/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/opt/conda/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython']


In [97]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [63]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files


# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [64]:
song_files = get_files('data/song_data')

In [65]:
filepath = song_files

In [69]:
df = pd.read_json(filepath[5], lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [70]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [71]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [72]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [73]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [74]:
log_files = get_files('data/log_data')

In [75]:
filepath = log_files

In [76]:
df =  pd.read_json(filepath[5], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,See No Evil (Remastered LP Version),200,1541381456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Blues To Bechet (LP Version),200,1541381694796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,It's My Job To Keep Punk Rock Elite,200,1541382040796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [77]:
df = df.loc[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,See No Evil (Remastered LP Version),200,1541381456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Blues To Bechet (LP Version),200,1541381694796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,It's My Job To Keep Punk Rock Elite,200,1541382040796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [78]:
df['ts'] = pd.to_datetime(df['ts'],unit='ms')
t = pd.DataFrame(df['ts'])
t.head()

,ts
0,2018-11-05 00:33:12.796
1,2018-11-05 01:27:22.796
2,2018-11-05 01:30:56.796
3,2018-11-05 01:34:54.796
4,2018-11-05 01:40:40.796


In [79]:
time_data = (t['ts'], t['ts'].dt.hour, t['ts'].dt.day, t['ts'].dt.weekofyear, t['ts'].dt.month, t['ts'].dt.year, t['ts'].dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [80]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-05 00:33:12.796,0,5,45,11,2018,0
1,2018-11-05 01:27:22.796,1,5,45,11,2018,0
2,2018-11-05 01:30:56.796,1,5,45,11,2018,0
3,2018-11-05 01:34:54.796,1,5,45,11,2018,0
4,2018-11-05 01:40:40.796,1,5,45,11,2018,0


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [81]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [88]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,44,Aleena,Kirby,F,paid
2,44,Aleena,Kirby,F,paid
3,44,Aleena,Kirby,F,paid
4,44,Aleena,Kirby,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [98]:
print(user_table_insert)
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()


    INSERT INTO dim_users 
    (user_id, firstName, lastName, gender, level) 
    VALUES (%s, %s, %s, %s, %s) 
    ON CONFLICT (user_id) DO UPDATE SET 
    level=users.level 


ProgrammingError: missing FROM-clause entry for table "users"
LINE 6:     level=users.level 
                  ^


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [96]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.